In [35]:
import math
import numpy as np
import pandas as pd
import plotly.express as px
from python_module.pricing_model import BSMModel, SABRModel, HestonHullWhiteModel


pd.options.display.max_rows = 30
pd.options.display.max_columns = 30
pd.options.display.float_format = '{:,.2f}'.format

In [36]:
def compute_backtest(price_ts, vol_target=0.3, option_type='call', strike_pct=1):
    
    bt_df = price_ts.to_frame('F')
    bt_df['K'] = bt_df['F'].iloc[0] * strike_pct
    bt_df['count'] = 1
    bt_df['days_to_maturity'] = bt_df['count'].iloc[::-1].cumsum().iloc[::-1] - 1
    bt_df['T'] = bt_df['days_to_maturity'] / 252

    rows = []
    for index, row in bt_df.iterrows():
        
        row_dict = row.to_dict()

        row_dict['date'] = index
        F = row_dict['F']
        K = row_dict['K']
        T = row_dict['T']

        pricing_results = BSMModel.compute_option_with_forward(
            F=F,
            K=K,
            T=T,
            r=0,
            sigma=vol_target,
            option_type=option_type,
            compute_greeks=True
            )
        merged_dict = {**row_dict, **pricing_results}
        rows.append(merged_dict)
    bt_df = pd.DataFrame(rows)
    bt_df['dP'] = bt_df['price'].diff()
    bt_df['dH'] = bt_df['F'].diff() * bt_df['delta'].shift(1)

    # Extra outputs
    bt_df['dP_cumsum'] = bt_df['dP'].cumsum()
    bt_df['dH_cumsum'] = bt_df['dH'].cumsum()
    bt_df['premium'] = bt_df['price'].iloc[0]
    bt_df['F0'] = bt_df['F'].iloc[0]
    return bt_df

In [37]:
# Inputs
vol_target = 0.2
rolling_window = 20

In [38]:
# Load data
df_init = pd.read_csv('data/SPY.csv', index_col=0, parse_dates=True)

# Compute the vol target price
df = df_init.copy()
df['log_return'] = np.log(df['price'] / df['price'].shift(1))
df['rolling_std'] = df['log_return'].rolling(window=rolling_window).std() * np.sqrt(252)
df['leverage'] = vol_target / df['rolling_std']
df = df.dropna()
df['vt_price'] = ((df['price'].pct_change() * df['leverage'].shift(1)).fillna(0).add(1).cumprod()) * 100

In [76]:
# Load data
df_init = pd.read_csv('data/SPY.csv', index_col=0, parse_dates=True)

# Compute the vol target price
df = df_init.copy()
df['returns'] = df['price'].pct_change().fillna(0)
df['volatility'] = df['returns'].rolling(250).std() * np.sqrt(252)
df = df.dropna()

In [ ]:
df['median'] = df['volatility'].describe()['50%']
df['top_percentitle'] = df['volatility'].describe()['75%']
df['above_median'] = (df['volatility'] > df['median']).astype(int)
df['above_top_percentitle'] = (df['volatility'] > df['top_percentitle']).astype(int)


In [85]:
df

,price,returns,volatility,median,top_percentitle,above_median,above_top_percentitle
2007-12-28,106.08,-0.00,0.16,0.16,0.20,1,0
2007-12-31,105.30,-0.01,0.16,0.16,0.20,1,0
2008-01-02,104.37,-0.01,0.16,0.16,0.20,1,0
2008-01-03,104.32,-0.00,0.16,0.16,0.20,1,0
2008-01-04,101.77,-0.02,0.16,0.16,0.20,1,0
...,...,...,...,...,...,...,...
2025-08-25,642.47,-0.00,0.20,0.16,0.20,1,0
2025-08-26,645.16,0.00,0.20,0.16,0.20,1,0
2025-08-27,646.63,0.00,0.20,0.16,0.20,1,0
2025-08-28,648.92,0.00,0.20,0.16,0.20,1,0


In [42]:
df['vt_change'] = df['vt_price'].pct_change().fillna(0)

In [50]:
df['fut_return'] = df['vt_change'].rolling(252).apply(lambda x: x.add(1).prod()-1 ).shift(-251)

In [52]:
px.scatter(df.set_index('leverage')['fut_return'])

In [54]:
px.line(df['leverage'])

In [44]:
df

,price,log_return,rolling_std,leverage,vt_price,vt_change
2007-02-01,102.26,0.01,0.08,2.45,100.00,0.00
2007-02-02,102.40,0.00,0.08,2.45,100.34,0.00
2007-02-05,102.43,0.00,0.07,2.69,100.41,0.00
2007-02-06,102.45,0.00,0.07,2.72,100.48,0.00
2007-02-07,102.68,0.00,0.07,2.74,101.09,0.01
...,...,...,...,...,...,...
2025-08-25,642.47,-0.00,0.12,1.65,"1,392.75",-0.01
2025-08-26,645.16,0.00,0.12,1.65,"1,402.39",0.01
2025-08-27,646.63,0.00,0.12,1.65,"1,407.67",0.00
2025-08-28,648.92,0.00,0.12,1.67,"1,415.92",0.01


In [14]:
price_ts = df['vt_price'].iloc[:253]
first_days = [df['vt_price'].index[df['vt_price'].index.year == year][0] for year in df['vt_price'].index.year.unique()]

In [33]:
global_results = dict()
for date in first_days:
    price_ts = df['vt_price'].loc[date:].iloc[:253]
    price_ts = price_ts.pct_change().fillna(0).add(1).cumprod()*100
    atm_call = compute_backtest(price_ts, vol_target=vol_target, option_type='call', strike_pct=1)
    up_call = compute_backtest(price_ts, vol_target=vol_target, option_type='call', strike_pct=1.2)
    atm_put = compute_backtest(price_ts, vol_target=vol_target, option_type='put', strike_pct=1)
    down_put = compute_backtest(price_ts, vol_target=vol_target, option_type='put', strike_pct=0.8)
    up_financing = compute_backtest(price_ts, vol_target=vol_target, option_type='call', strike_pct=1.4)
    down_financing = compute_backtest(price_ts, vol_target=vol_target, option_type='call', strike_pct=1.35)
    total_premium = atm_put.iloc[0]['premium']+atm_call.iloc[0]['premium']-up_call.iloc[0]['premium']-down_put.iloc[0]['premium']
    max_financing_q = 90 / (up_financing.iloc[0]['K']-down_financing.iloc[0]['K'])
    financing_premium = down_financing.iloc[0]['premium']-up_financing.iloc[0]['premium']
    financing_q = min([total_premium / financing_premium, max_financing_q])
    strategy_cost = total_premium-(financing_premium*financing_q)
    financing_dh = (-down_financing['dH_cumsum'] + up_financing['dH_cumsum']) * financing_q
    strategy_cumsum_pnl = atm_call['dH_cumsum'] + atm_put['dH_cumsum'] - up_call['dH_cumsum'] - down_put['dH_cumsum'] + financing_dh
    strategy_cumsum_pnl_f = strategy_cumsum_pnl.loc[abs(strategy_cumsum_pnl) >= 30]
    bt_result = strategy_cumsum_pnl.describe().to_dict()
    if strategy_cumsum_pnl_f.shape[0] > 0:
        bt_result['pnl'] = strategy_cumsum_pnl_f.iloc[0]
    else:
        bt_result['pnl'] = strategy_cumsum_pnl.iloc[-1]
    global_results[date] = bt_result

In [34]:
px.scatter(pd.DataFrame(global_results).transpose()['pnl'])

In [11]:
strategy_cumsum_pnl_f = strategy_cumsum_pnl.loc[abs(strategy_cumsum_pnl) >= 10]

In [ ]:
strategy_cumsum_pnl_f

51    -11.67
52    -12.66
53    -12.89
54    -12.14
55    -12.26
       ...  
248    13.71
249    14.70
250    15.24
251    16.17
252    14.52
Name: dH_cumsum, Length: 120, dtype: float64

In [157]:
price_ts = df['vt_price'].loc['2024-02-27':].iloc[:253]
price_ts = price_ts.pct_change().fillna(0).add(1).cumprod()*100
atm_call = compute_backtest(price_ts, vol_target=vol_target, option_type='call', strike_pct=1)
up_call = compute_backtest(price_ts, vol_target=vol_target, option_type='call', strike_pct=1.2)
atm_put = compute_backtest(price_ts, vol_target=vol_target, option_type='put', strike_pct=1)
down_put = compute_backtest(price_ts, vol_target=vol_target, option_type='put', strike_pct=0.8)
up_financing = compute_backtest(price_ts, vol_target=vol_target, option_type='call', strike_pct=1.3)
down_financing = compute_backtest(price_ts, vol_target=vol_target, option_type='call', strike_pct=1.25)
total_premium = atm_put.iloc[0]['premium']+atm_call.iloc[0]['premium']-up_call.iloc[0]['premium']-down_put.iloc[0]['premium']
max_financing_q = 90 / (up_financing.iloc[0]['K']-down_financing.iloc[0]['K'])
financing_premium = down_financing.iloc[0]['premium']-up_financing.iloc[0]['premium']
financing_q = min([total_premium / financing_premium, max_financing_q])
strategy_cost = total_premium-(financing_premium*financing_q)
financing_dh = (-down_financing['dH_cumsum'] + up_financing['dH_cumsum']) * financing_q
strategy_cumsum_pnl = atm_call['dH_cumsum'] + atm_put['dH_cumsum'] - up_call['dH_cumsum'] - down_put['dH_cumsum'] + financing_dh
if strategy_cumsum_pnl.max() > 20:
    early_stop_index = strategy_cumsum_pnl.loc[strategy_cumsum_pnl > 20].index[0]
    early_stop_strategy = strategy_cumsum_pnl.loc[:early_stop_index]
else:
    early_stop_strategy = strategy_cumsum_pnl
bt_result = early_stop_strategy.describe().to_dict()
bt_result['pnl'] = early_stop_strategy.iloc[-1]

In [165]:
financing_delta = (-down_financing['delta'] + up_financing['delta']) * financing_q
strategy_delta = atm_call['delta'] + atm_put['delta'] - up_call['delta'] - down_put['delta'] + financing_delta

In [166]:
px.line(strategy_delta)

In [158]:
abs(price_ts.iloc[-1]-price_ts.iloc[0]), early_stop_strategy.iloc[-1]

(29.454550319003204, -48.85104623720173)

In [160]:
px.line(price_ts)

In [162]:
strategy_cumsum_pnl.index = price_ts.index

In [163]:
px.line(strategy_cumsum_pnl)

In [132]:
pd.DataFrame(global_results)

2024-08-27  2024-08-26  2024-08-23  2024-08-22  2024-08-21  2024-08-20  \
count      252.00      252.00      252.00      252.00      252.00      252.00   
mean         3.98        3.83        4.02        2.98        3.74        3.41   
std          9.52        9.59        9.45        9.97        9.56        9.71   
min        -16.88      -17.13      -16.74      -18.56      -17.19      -17.76   
25%         -2.93       -3.11       -2.75       -4.31       -3.07       -3.53   
50%          4.69        4.22        4.78        2.89        3.60        3.14   
75%         12.48       12.30       12.42       12.17       12.60       12.38   
max         18.74       18.69       18.80       18.36       18.72       18.59   
pnl         14.52       16.60       15.16       16.46       13.65       15.47   

       2024-08-19  2024-08-16  2024-08-15  2024-08-14  2024-08-13  2024-08-12  \
count      252.00      252.00      252.00      248.00      249.00      244.00   
mean         3.53        2.66        2.44        0.39        0.09       -2.50   
std          9.63       10.08       10.20       10.92       11.09       11.46   
min        -17.51      -19.06      -19.44      -22.31      -22.87      -25.75   
25%         -3.28       -4.62       -4.90       -7.58       -7.98      -10.67   
50%          3.22        1.96        1.69       -0.73       -1.20       -4.07   
75%         12.30       12.00       11.84       10.48       10.19        8.67   
max         18.67       18.29       18.20       20.06       21.61       22.53   
pnl         10.46       13.42       14.83       20.06       21.61       22.53   

       2024-08-09  2024-08-08  2024-08-07  2024-08-06  2024-08-05  2024-08-02  \
count      245.00      245.00      246.00      247.00      248.00      252.00   
mean        -2.52       -3.20       -7.11       -5.79       -7.43       -1.01   
std         11.49       11.57       12.71       12.41       13.00       11.35   
min        -25.88      -26.70      -31.16      -29.91      -31.78      -24.80   
25%        -10.73      -11.48      -16.83      -14.95      -17.40       -9.30   
50%         -4.10       -4.86       -9.03       -7.83       -9.32       -2.19   
75%          8.63        8.18        5.67        6.60        5.40        9.60   
max         23.45       21.29       31.35       31.92       39.16       19.02   
pnl         23.45       21.29       31.35       31.92       39.16       19.02   

       2024-08-01  2024-07-31  2024-07-30  2024-07-29  2024-07-26  2024-07-25  \
count      252.00      252.00      252.00      252.00      252.00      252.00   
mean         2.12        4.26        1.79        2.68        2.57        0.60   
std         10.28        9.44       10.50       10.21       10.26       11.04   
min        -20.18      -16.59      -20.81      -19.43      -19.61      -22.79   
25%         -5.15       -2.16       -5.60       -4.34       -4.45       -7.18   
50%          1.57        3.77        0.84        1.80        1.61       -0.84   
75%         11.59       13.12       11.65       12.11       12.04       11.05   
max         18.41       19.36       18.34       18.72       18.69       17.84   
pnl          8.20        8.56        4.48       13.52       16.02       15.76   

       2024-07-24  2024-07-23  2024-07-22  2024-07-19  2024-07-18  2024-07-17  \
count      252.00      176.00      177.00      252.00      179.00      180.00   
mean         1.56        2.85        3.22        4.67        2.51        4.58   
std         10.68        7.28        7.17        9.16        7.35        6.80   
min        -21.27      -12.51      -11.97      -15.71      -13.08       -9.97   
25%         -5.86       -1.87       -1.37       -1.40       -1.66        0.81   
50%          0.35        3.32        3.58        3.82        2.96        5.02   
75%         11.48        6.73        6.96       13.21        6.37        8.11   
max         19.41       20.22       20.35       19.98       20.19       20.88   
pnl         19.33       20.22     

In [130]:
px.line(pd.DataFrame(global_results).loc['pnl'])

In [ ]:
T = 1
F = 100

call_100 = BSMModel.compute_option_with_forward(F=F, K=100, T=T, r=0, sigma=0.3, option_type='call', compute_greeks=True)
call_120 = BSMModel.compute_option_with_forward(F=F, K=120, T=T, r=0, sigma=0.3, option_type='call', compute_greeks=True)

put_100 = BSMModel.compute_option_with_forward(F=F, K=100, T=T, r=0, sigma=0.3, option_type='put', compute_greeks=True)
put_80 = BSMModel.compute_option_with_forward(F=F, K=80, T=T, r=0, sigma=0.3, option_type='put', compute_greeks=True)

call_125 = BSMModel.compute_option_with_forward(F=F, K=125, T=T, r=0, sigma=0.3, option_type='call', compute_greeks=True)
call_130 = BSMModel.compute_option_with_forward(F=F, K=130, T=T, r=0, sigma=0.3, option_type='call', compute_greeks=True)

put_75 = BSMModel.compute_option_with_forward(F=F, K=75, T=T, r=0, sigma=0.3, option_type='put', compute_greeks=True)
put_70 = BSMModel.compute_option_with_forward(F=F, K=70, T=T, r=0, sigma=0.3, option_type='put', compute_greeks=True)


In [233]:
call_100['price']-call_120['price']+put_100['price']-put_80['price']-9*put_75['price']+9*put_70['price']-8*call_125['price']+8*call_130['price']

0.08159365607887281

In [234]:
call_100['delta']-call_120['delta']+put_100['delta']-put_80['delta']-9*put_75['delta']+8*put_70['delta']-8*call_125['delta']+8*call_130['delta']

0.12690359799993178

In [235]:
grid = dict()
for F in range(50, 150, 10):
    for T in [252/252, 126/252, 63/252, 21/252, 7/252, 1/252]:

        call_100 = BSMModel.compute_option_with_forward(F=F, K=100, T=T, r=0, sigma=0.3, option_type='call', compute_greeks=True)
        call_120 = BSMModel.compute_option_with_forward(F=F, K=120, T=T, r=0, sigma=0.3, option_type='call', compute_greeks=True)

        put_100 = BSMModel.compute_option_with_forward(F=F, K=100, T=T, r=0, sigma=0.3, option_type='put', compute_greeks=True)
        put_80 = BSMModel.compute_option_with_forward(F=F, K=80, T=T, r=0, sigma=0.3, option_type='put', compute_greeks=True)

        call_125 = BSMModel.compute_option_with_forward(F=F, K=125, T=T, r=0, sigma=0.3, option_type='call', compute_greeks=True)
        call_130 = BSMModel.compute_option_with_forward(F=F, K=130, T=T, r=0, sigma=0.3, option_type='call', compute_greeks=True)

        put_75 = BSMModel.compute_option_with_forward(F=F, K=75, T=T, r=0, sigma=0.3, option_type='put', compute_greeks=True)
        put_70 = BSMModel.compute_option_with_forward(F=F, K=70, T=T, r=0, sigma=0.3, option_type='put', compute_greeks=True)

        premium = call_100['price']-call_120['price']+put_100['price']-put_80['price']-9*put_75['price']+9*put_70['price']-8*call_125['price']+8*call_130['price']
        delta = call_100['delta']-call_120['delta']+put_100['delta']-put_80['delta']-9*put_75['delta']+8*put_70['delta']-8*call_125['delta']+8*call_130['delta']
        grid[(F, T)] = dict(premium=premium, delta=delta)

In [257]:
grid = dict()
for F in range(50, 150, 10):
    for T in [252/252, 126/252, 63/252, 21/252, 7/252, 1/252]:

        call_100 = BSMModel.compute_option_with_forward(F=F, K=100, T=T, r=0, sigma=0.3, option_type='call', compute_greeks=True)
        put_100 = BSMModel.compute_option_with_forward(F=F, K=100, T=T, r=0, sigma=0.3, option_type='put', compute_greeks=True)

        premium = call_100['price']+put_100['price']
        delta = call_100['delta']+put_100['delta']
        grid[(F, T)] = dict(premium=premium, delta=delta)

In [258]:
grid = pd.DataFrame(grid).transpose().reset_index()

In [259]:
grid.columns = ['F', 'T', 'premium', 'delta']

In [260]:
px.line(grid[grid['T']==1].set_index('F')['delta'])

In [262]:
px.line(grid[grid['T']==0.5].set_index('F')['delta'])

In [263]:
px.line(grid[grid['T']==0.25].set_index('F')['premium'])

In [254]:
grid

,F,T,premium,delta
0,50,1.00,-21.62,1.24
1,50,0.50,-23.61,1.22
2,50,0.25,-24.74,1.08
3,50,0.08,-25.00,1.00
4,50,0.03,-25.00,1.00
...,...,...,...,...
55,140,0.50,-7.88,-0.39
56,140,0.25,-9.70,-0.54
57,140,0.08,-14.09,-0.73
58,140,0.03,-18.57,-0.44


In [244]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Surface(z=grid['delta'].values.reshape(len(grid['F'].unique()), len(grid['T'].unique())),
                                 x=grid['F'].unique(),
                                 y=grid['T'].unique())])

fig.update_layout(scene = dict(
                    xaxis_title='F',
                    yaxis_title='T',
                    zaxis_title='Premium'),
                  title='Premium Surface Plot')

fig.show()

In [224]:
call_100['delta']-call_120['delta']+put_100['delta']-put_80['delta']-10*put_75['delta']+10*put_70['delta']-10*call_125['delta']+10*call_130['delta']

-0.0038746084155518723

In [217]:
long_call_spread = atm_call['price'] - up_call['price']
long_put_spread = atm_put['price'] - down_put['price']
short_call_spread = (call_up_financing['price'] - call_down_financing['price']) * 10
short_put_spread = (put_up_financing['price'] - put_down_financing['price']) * 10

In [220]:
long_call_spread+long_put_spread

14.872123377108931

In [218]:
short_put_spread

-41.06823243472178

In [219]:
short_call_spread

-8.439923640349534

In [202]:
delta = atm_call['delta'] + atm_put['delta'] - up_call['delta'] - down_put['delta']
financing_delta = (up_financing['delta'] - down_financing['delta']) * 17

In [203]:
delta+financing_delta

-2.0422496944057995